# torchtextを用いたDataset, DataLoaderの実装
torchtext: PyTorchの自然言語処理パッケージ

## torchtextのインストール
``` bash
pip install torchtext
```

## 使用するデータ
- text_train.tsv
- text_val.tsv
- text_test.tsv

実際に学習をするわけではないので，上記3つはすべて同じで，4行しかない  

In [1]:
import os

In [2]:
data_dir = "../../datasets/ptca_datasets/chapter7"

In [3]:
with open(os.path.join(data_dir, "text_train.tsv"), encoding='utf-8') as f:
    for line in f.readlines():
        print(line.strip())

王と王子と女王と姫と男性と女性がいました。	0
機械学習が好きです。	1
本章から自然言語処理に取り組みます。	1
本章では商品レビューの短い文章に対して、その文章がネガティブな評価をしている文章なのか、ポジティブな評価をしている文章なのか、2値のクラス分類する分類モデルを構築します。	0


文章は特に意味がないが，0はネガティブ，1はポジティブのラベルになっている

## 前処理と単語分割の関数を実装
まずはJanomeを使用

In [4]:
from janome.tokenizer import Tokenizer

j_t = Tokenizer()

def tokenizer_janome(text):
    return j_t.tokenize(text, wakati=True)

前処理は
- 今回行わないこと
    - 全角・半角の統一
    - 英語の小文字化
    - 記号と数字の除去
    - 特定文字を正規表現で置換
- 今回行うこと
    - [改行](https://ja.stackoverflow.com/questions/12897/%E6%94%B9%E8%A1%8C%E3%81%AE-n%E3%81%A8-r-n%E3%81%AE%E9%81%95%E3%81%84%E3%81%AF%E4%BD%95%E3%81%A7%E3%81%99%E3%81%8B)，半角スペース，全角スペースを削除
    - 数字はすべて0にする

In [5]:
import re

def preprocessing_text(text):
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub(' ', '', text)
    text = re.sub('　', '', text)
    text = re.sub(r'[0-9 ０-９]', '0', text)
    return text

def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_janome(text)
    return ret

In [6]:
text = "昨日は とても暑く，気温が36度もあった。"
print(tokenizer_with_preprocessing(text))

['昨日', 'は', 'とても', '暑く', '，', '気温', 'が', '00', '度', 'も', 'あっ', 'た', '。']


## 文章データの読み込み
torchtext.data.Fieldでフィールドごとのテキストの読み込み方を指定  
引数には以下がある
- sequential: データの長さが可変か？今回はTrue
- tokenize: 前処理，単語分割をするための関数を指定
- use_vocab: 単語をボキャブラリー(単語集)に追加するかを設定
- lower: アルファベットを小文字に変換するかを指定
- include_length: 文章の単語数のデータを保持するかの設定
- batch_first: ミニバッチの次元を先頭に用意するかを指定
- fix_length: すべての文章を，指定した長さと同じになるようpaddingする

In [7]:
import torchtext

max_length = 25
TEXT = torchtext.data.Field(
    sequential=True,
    tokenize=tokenizer_with_preprocessing,
    use_vocab=True,
    lower=True,
    include_lengths=True,
    batch_first=True,
    fix_length=max_length
)

LABEL = torchtext.data.Field(
    sequential=False,
    use_vocab=False
)

data.TabularDatasetで文章データを読み込む  
train,validation,testの3つのファイルを指定できる他，fieldsで各列に対して個別の処理を行うことができる．  
出力される辞書のキーを一緒に指定できる．

In [8]:
train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path=data_dir,
    train='text_train.tsv',
    validation='text_val.tsv',
    test='text_test.tsv',
    format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)]
)

print('訓練データの数', len(train_ds))
print('訓練データ[0]', vars(train_ds[0]))
print('訓練データ[1]', vars(train_ds[1]))

訓練データの数 4
訓練データ[0] {'Text': ['王', 'と', '王子', 'と', '女王', 'と', '姫', 'と', '男性', 'と', '女性', 'が', 'い', 'まし', 'た', '。'], 'Label': '0'}
訓練データ[1] {'Text': ['機械', '学習', 'が', '好き', 'です', '。'], 'Label': '1'}


なお,[vars](https://docs.python.org/ja/3/library/functions.html#vars)はオブジェクトを辞書として返すため，`__dict__`メソッドを呼び出す組み込み関数

## 単語の数値化
本節では，単語にIDを振ることで数値化する．  
IDを振るには，ボキャブラリという単語の集まりを用意する．  
全ての単語にIDを振るのではなく，扱う対象を絞ることが多い．  
torchtext.data.Fieldインスタンスでボキャブラリを構築できる．

In [12]:
# 訓練データからmin_freq以上の頻度の単語を使用してボキャブラリを構築
TEXT.build_vocab(train_ds, min_freq=1)

# メンバ変数にvocabが追加されている
# 訓練データないの単語を頻度を出力
print(TEXT.vocab.freqs, "\n")

# どの単語がどのIDに割り振られたのかを確認
print(TEXT.vocab.stoi)

Counter({'と': 5, '。': 4, 'の': 4, '文章': 4, 'な': 4, 'が': 3, '、': 3, 'を': 3, 'し': 3, '本章': 2, 'ます': 2, '評価': 2, 'て': 2, 'いる': 2, 'か': 2, '分類': 2, '王': 1, '王子': 1, '女王': 1, '姫': 1, '男性': 1, '女性': 1, 'い': 1, 'まし': 1, 'た': 1, '機械': 1, '学習': 1, '好き': 1, 'です': 1, 'から': 1, '自然': 1, '言語': 1, '処理': 1, 'に': 1, '取り組み': 1, 'で': 1, 'は': 1, '商品': 1, 'レビュー': 1, '短い': 1, 'に対して': 1, 'その': 1, 'ネガティブ': 1, 'ポジティブ': 1, '0': 1, '値': 1, 'クラス': 1, 'する': 1, 'モデル': 1, '構築': 1}) 

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f9aad683da0>>, {'<unk>': 0, '<pad>': 1, 'と': 2, '。': 3, 'な': 4, 'の': 5, '文章': 6, '、': 7, 'が': 8, 'し': 9, 'を': 10, 'いる': 11, 'か': 12, 'て': 13, 'ます': 14, '分類': 15, '本章': 16, '評価': 17, '0': 18, 'い': 19, 'から': 20, 'する': 21, 'その': 22, 'た': 23, 'で': 24, 'です': 25, 'に': 26, 'に対して': 27, 'は': 28, 'まし': 29, 'クラス': 30, 'ネガティブ': 31, 'ポジティブ': 32, 'モデル': 33, 'レビュー': 34, '値': 35, '処理': 36, '取り組み': 37, '商品': 38, '女性': 39, '女王': 40, '好き': 41, '姫': 42, '学習': 43, '構築':

ここで，`<unk>`はunknownのことで，ボキャブラリにない単語を意味する．  
`<pad>`は長さの統一のために使われるpaddingのこと

## DataLoaderの作成
torchtext.data.Iteratorによってミニバッチサイズを指定したDataLoaderを作成

In [13]:
train_dl = torchtext.data.Iterator(
    train_ds, batch_size=2
)

val_dl = torchtext.data.Iterator(
    val_ds, batch_size=2, train=False, sort=False
)

test_dl = torchtext.data.Iterator(
    test_ds, batch_size=2, train=False, sort=False
)

batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)

(tensor([[46,  2, 47,  2, 40,  2, 42,  2, 48,  2, 39,  8, 19, 29, 23,  3,  1,  1,
          1,  1,  1,  1,  1,  1,  1],
        [45, 43,  8, 41, 25,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1]]), tensor([16,  6]))
tensor([0, 1])


単語はIDに置き換えられており，25単語よりも短い文章は`<pad>＝1`が入っている

しかし，単語はIDではなくベクトルで扱いたいので，その方法を次節で説明